In [1]:
import openeo
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
# Establish the connetion to openeo
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

In [3]:
# Set up the region of interest
vent_location = vlat, vlon = [38.789, 15.213]
clear_locations = [
    [38.789, 16.213],
    [39.589, 16.213],
    [39.589, 15.213],
    [39.589, 14.213],
    [38.789, 14.213]
]
bounds = lonW, lonE, latS, latN = [13.7, 16.6, 38, 40]
radius = 25000

start_date = '2018-01-01'
end_date = '2020-01-01'

In [ ]:
# Define the datacube to acquire
so2_cube = connection.load_collection(
    "SENTINEL_5P_L2",
    temporal_extent=(start_date, end_date),
    spatial_extent={"west": lonW, "south": latS, "east": lonE, "north": latN, "crs": "EPSG:4326"},
    bands=["SO2"]
)

# Now aggregate by day to avoid having multiple data per day
so2_cube = so2_cube.aggregate_temporal_period(reducer="mean", period="day")

job = so2_cube.execute_batch(title="SO2 from Stromboli", outputfile="stromboli_so2_data.nc")

0:00:00 Job 'j-2511111534394f2492a531b1197f0af2': send 'start'
0:00:13 Job 'j-2511111534394f2492a531b1197f0af2': created (progress 0%)
0:00:18 Job 'j-2511111534394f2492a531b1197f0af2': created (progress 0%)
0:00:24 Job 'j-2511111534394f2492a531b1197f0af2': created (progress 0%)
0:00:32 Job 'j-2511111534394f2492a531b1197f0af2': running (progress N/A)
0:00:42 Job 'j-2511111534394f2492a531b1197f0af2': running (progress N/A)
0:00:54 Job 'j-2511111534394f2492a531b1197f0af2': running (progress N/A)
0:01:10 Job 'j-2511111534394f2492a531b1197f0af2': running (progress N/A)


In [ ]:
def haversine(start_coords, end_coords, radius=6371000):
    """Calculate the distance between two points.

    Parameters
    ----------
    start_coords : tuple
        Start coordinates (lat, lon) in decimal degrees (+ve = north/east).
    end_coords : tuple
        End coordinates (lat, lon) in decimal degrees (+ve = north/east).
    radius: float, optional
        Radius of the body in meters. Default is set to the Earth radius
        (6731km).

    Returns
    -------
    distance : float
        The linear distance between the two points in meters.
    """
    # Unpack the coordinates and convert to radians
    lat1, lon1 = np.radians(start_coords)
    lat2, lon2 = np.radians(end_coords)

    # Calculate the change in lat and lon
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Calculate the square of the half chord length
    a = np.add(
        (np.sin(dlat/2))**2,
        np.cos(lat1) * np.cos(lat2) * (np.sin(dlon/2))**2
    )

    # Calculate the angular distance
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    # Find distance moved
    distance = radius * c

    return distance

In [ ]:
# Open the file
with xr.open_dataset('stromboli_so2_data.nc') as ds:

    # Define the X and Y grid in 2D
    X, Y = np.meshgrid(ds.x, ds.y)

    # Calculate the region mask for the volcano
    region_mask = (
        haversine([vlat, vlon], [Y.ravel(), X.ravel()]) <= radius
    ).reshape(X.shape)

    # Add the clear regions to the mask
    for i, (clat, clon) in enumerate(clear_locations):
        region_mask = np.where(
            (haversine([clat, clon], [Y.ravel(), X.ravel()])).reshape(X.shape) <= radius,
            i+1, region_mask
        )

    # Initialise the output arrays in a dictionary
    output = {
        'volcano': np.full(len(ds.t), np.nan),
        **{
            f'clear_{i+1}': np.full(len(ds.t), np.nan)
            for i in range(len(clear_locations))
        }
    }

    # For each day, extract the region SO2 VCD and sum the values
    for ti in range(len(ds.t)):
        day_so2 = ds.SO2.isel(t=ti)
        output['volcano'][ti] = np.nansum(np.where(region_mask==1, day_so2, 0))
        for i in range(len(clear_locations)):
            output[f'clear_{i+1}'][ti] = np.nansum(np.where(region_mask==i+1, day_so2, 0))

# Convert to a dataframe
df = pd.DataFrame({'date': ds.t, **output})

In [ ]:
fig, ax = plt.subplots(figsize=[7, 3])

ax.plot(df['date'], np.cumsum(df['volcano']), label='Voclano', color='k', lw=2)
for i in range(len(clear_locations)):
    ax.plot(df['date'], np.cumsum(df[f'clear_{i+1}']), f'C{i}-', alpha=0.5, label=f'Clear {i+1}')

ax.legend()

locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

plt.show()